# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
city_data_csv = "../WeatherPy/output_data/clean_city_data.csv"
# Create dataframe
city_data = pd.read_csv(city_data_csv)
city_data

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,beloha,-25.1667,45.0500,66.02,98,18,4.16,MG,1665435071
1,sarangani,5.4033,125.4636,83.44,77,75,13.53,PH,1665435072
2,tuktoyaktuk,69.4541,-133.0374,30.20,86,100,20.71,CA,1665434770
3,arman,59.7000,150.1667,24.17,82,45,5.84,RU,1665435074
4,biankouma,7.7391,-7.6138,70.34,91,13,1.79,CI,1665435074
...,...,...,...,...,...,...,...,...,...
525,laizhou,37.1717,119.9214,49.21,54,0,6.87,CN,1665436272
526,harper,4.3750,-7.7169,76.77,89,83,5.39,LR,1665436272
527,sangar,63.9242,127.4739,16.16,93,46,1.95,RU,1665436272
528,mareeba,-17.0000,145.4333,72.32,92,11,2.51,AU,1665436272


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [70]:
gmaps.configure(api_key=g_key)

city_data = city_data.dropna()

locations = city_data[["Lat", "Lng"]].astype(float)
locations
humidity = city_data["Humidity"].astype(float)
humidity

fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)
layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'}

heat_layer = gmaps.heatmap_layer(locations, weights=humidty, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
narrowed_cities = city_data.loc[(city_data["Max_Temp"] < 80) 
                                    & (city_data["Max_Temp"] > 70) 
                                    & (city_data["Wind Speed"] < 10) 
                                    & (city_data["Cloudiness"] == 0),:].dropna()
narrowed_cities.reset_index(inplace=True)
del narrowed_cities["index"]
narrowed_cities

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,cape town,-33.9258,18.4232,70.38,67,0,3.00,ZA,1665434391
1,mahajanga,-15.7167,46.3167,73.42,78,0,2.30,MG,1665435196
2,alanya,36.5438,31.9998,73.08,50,0,4.61,TR,1665435402
3,jalu,29.0331,21.5482,72.64,41,0,6.53,LY,1665435465
4,ashland,40.8167,-82.2665,70.16,44,0,7.34,US,1665435735
5,kumluca,36.3703,30.2869,72.37,56,0,4.63,TR,1665435850
6,hamilton,39.1834,-84.5333,75.07,37,0,9.22,US,1665435867
7,morondava,-20.2833,44.2833,75.45,84,0,7.23,MG,1665435915
8,rawatsar,29.2833,74.3833,73.54,85,0,4.43,IN,1665435977
9,marsa matruh,31.3525,27.2453,71.53,66,0,7.74,EG,1665435929


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [58]:
hotel_df = narrowed_cities 

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    params["location"] = f'{lat},{lng}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    
    
    response = requests.get(base_url, params=params)
    response = response.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
hotel_df    



,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,cape town,-33.9258,18.4232,70.38,67,0,3.00,ZA,1665434391,The Bay Hotel
1,mahajanga,-15.7167,46.3167,73.42,78,0,2.30,MG,1665435196,Baobab Tree Hotel & Spa
2,alanya,36.5438,31.9998,73.08,50,0,4.61,TR,1665435402,Harmony butik hotel&cafe
3,jalu,29.0331,21.5482,72.64,41,0,6.53,LY,1665435465,إستراحة صلاح الصادق
4,ashland,40.8167,-82.2665,70.16,44,0,7.34,US,1665435735,Hampton Inn & Suites Ashland
5,kumluca,36.3703,30.2869,72.37,56,0,4.63,TR,1665435850,Truva Pansiyon
6,hamilton,39.1834,-84.5333,75.07,37,0,9.22,US,1665435867,Graduate Cincinnati
7,morondava,-20.2833,44.2833,75.45,84,0,7.23,MG,1665435915,Palissandre Cote Ouest resort & SPA
8,rawatsar,29.2833,74.3833,73.54,85,0,4.43,IN,1665435977,Hotel Yuvraj AND RESTAURANT
9,marsa matruh,31.3525,27.2453,71.53,66,0,7.74,EG,1665435929,Beau Site


In [59]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [69]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))